# HW 1: Intrinsic Evaluation of Word Embeddings
**Due: February 16, 9:00 AM**

In this homework assignment, you will implement the 3CosAdd evaluation method<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) for word embeddings described in Section 4 and Subsection 4.1 of [Mikolov et al. (2013)](https://arxiv.org/abs/1301.3781). Please read those sections of the paper now if you have not seen them before.

The goal of 3CosAdd is to determine whether word embedding spaces exhibit algebraic relationships between analogous pairs of words. For instance, if we assume that the word _man_ has the same semantic relationship to the word _king_ as _woman_ does to _queen_ (in analogy notation, man : king :: woman : queen), then we might expect that
$$\overrightarrow{\text{king}} - \overrightarrow{\text{man}} \approx \overrightarrow{\text{queen}} - \overrightarrow{\text{woman}}$$
where $\overrightarrow{w}$ is the word embedding for word $w$. 

To measure the extent to which such relations exist, we use a test set that contains a number of analogies $w_1 : w_2 \mathrel{::} w_3 : w_4$. For each analogy, we ask the following _analogy question_: given $w_1$, $w_2$, and $w_3$, if $w_1 : w_2 \mathrel{::} w_3 : x$, then what is $x$? We consider the analogy question to have been _answered correctly_ if $\overrightarrow{w_4}$ happens to be the word embedding vector with the greatest cosine similarity to $\overrightarrow{w_2} - \overrightarrow{w_1} + \overrightarrow{w_3}$. The quality of the embedding space is then measured by its _analogy question accuracy_, given by the proportion of analogy questions that have been answered correctly.

## Important: Read Before Starting

In the following exercises, you will need to implement functions defined in the `embeddings` and `test_analogies` modules. **Please write all your code in the respective** `.py` **files for those two modules.** You should not submit this notebook with your solutions, and we will not grade it if you do. Please be aware that code written in a Jupyter notebook may run differently when copied into Python modules.

The outputs shown in this notebook are the outputs that you should get **when all problems have been completed correctly**. You may obtain different results if you attempt to run the code cells before you have completed the problem set, or if you have completed one or more problems incorrectly. **Obtaining the outputs shown in the code does not guarantee that your code is correct.**

To begin, please run the following `import` statements.

In [1]:
import itertools
from typing import Dict

import numpy as np

from embeddings import Embeddings
from test_analogies import cosine_sim, get_closest_words, load_analogies, \
    run_analogy_test

## Problem 1: Load Embeddings (4 Points + 1 EC in Total)

In this problem, you will implement the `embeddings.Embeddings` class, which is a container that holds a set of word embeddings. 

### Problem 1a: Inspect Class Definition and Data Files (No Submission, 0 Points)

Please look at the existing code in `embeddings.py`. Take note of the data attributes and methods that have been defined. What is the purpose of the following attributes?
* `words`
* `indices`
* `vectors` 
* `__len__`
* `__contains__`

Please also look at the three files `glove_50d.txt`, `glove_100d.txt`, and `glove_200d.txt` in the `data` folder. These files contain GloVe embeddings [(Pennington et al., 2014)](https://aclanthology.org/D14-1162/) of 50, 100, and 200 dimensions, trained on Wikipedia articles and the [Gigaword corpus](https://catalog.ldc.upenn.edu/LDC2011T07).<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2) What is the format of these files?

### Problem 1b: Implement the Embeddings Class (Code, 4 Points)

The `Embeddings` class contains two methods that are not implemented: the class method `from_file` and the magic method `__getitem__`. Please implement those two methods, making sure to follow the specifications given in their docstrings. When both methods have been implemented, the `Embeddings` class needs to support the following functionalities.

In [2]:
# Load embeddings from a file
embeddings = Embeddings.from_file("data/glove_200d.txt")

In [3]:
# Retrieve the number of vectors in the embeddings object
len(embeddings)

5485

In [4]:
# Check whether or not a given word has a word embedding
print("the" in embeddings)
print("lx496" in embeddings)

True
False


In [5]:
# Index embeddings by a list
vecs = embeddings[["the", "of"]]
print("vecs is an {} of shape {}.".format(type(vecs).__name__, vecs.shape))
print("Embedding for 'the': {}".format(vecs[0]))
print("Embedding for 'of': {}".format(vecs[1]))

vecs is an ndarray of shape (2, 200).
Embedding for 'the': [-7.1549e-02  9.3459e-02  2.3738e-02 -9.0339e-02  5.6123e-02  3.2547e-01
 -3.9796e-01 -9.2139e-02  6.1181e-02 -1.8950e-01  1.3061e-01  1.4349e-01
  1.1479e-02  3.8158e-01  5.4030e-01 -1.4088e-01  2.4315e-01  2.3036e-01
 -5.5339e-01  4.8154e-02  4.5662e-01  3.2338e+00  2.0199e-02  4.9019e-02
 -1.4132e-02  7.6017e-02 -1.1527e-01  2.0060e-01 -7.7657e-02  2.4328e-01
  1.6368e-01 -3.4118e-01 -6.6070e-02  1.0152e-01  3.8232e-02 -1.7668e-01
 -8.8153e-01 -3.3895e-01 -3.5481e-02 -5.5095e-01 -1.6899e-02 -4.3982e-01
  3.9004e-02  4.0447e-01 -2.5880e-01  6.4594e-01  2.6641e-01  2.8009e-01
 -2.4625e-02  6.3302e-01 -3.1700e-01  1.0271e-01  3.0886e-01  9.7792e-02
 -3.8227e-01  8.6552e-02  4.7075e-02  2.3511e-01 -3.2127e-01 -2.8538e-01
  1.6670e-01 -4.9707e-03 -6.2714e-01 -2.4904e-01  2.9713e-01  1.4379e-01
 -1.2325e-01 -5.8178e-02 -1.0290e-03 -8.2126e-02  3.6935e-01 -5.8442e-04
  3.4286e-01  2.8426e-01 -6.8599e-02  6.5747e-01 -2.9087e-02  1.6

**Hints:** 
* `__getitem__` can be implemented using at most 1 line of code.
* `from_file` can be implemented using at most 4 lines of code.
* Some of the above functionalities are already implemented in the starter code.
* Consider using the [`np.fromstring` function in NumPy](https://numpy.org/doc/stable/reference/generated/numpy.fromstring.html).
* Please consult one or more of the following resources if you are confused by the Python terminology in this problem: 
 * [Official Python Tutorial on Classes](https://docs.python.org/3/tutorial/classes.html)
 * [RealPython Tutorial on Object-Oriented Programming](https://realpython.com/python3-object-oriented-programming)
 * [RealPython Tutorial on Instance, Class, and Static Methods](https://realpython.com/instance-class-and-static-methods-demystified/)


### Problem 1c: Extra Credit (Written, 1 Point Extra Credit)

Look at the following two lines of code. The first line of code prints the word embeddings $\overrightarrow{\text{the}}$ and $\overrightarrow{\text{of}}$. The second line of code looks similar to the first line of code, but it does not work as intended. Why is this? 

**Note:** Depending on how you've implemented the two blank methods in `Embeddings`, it's possible that your code may not behave as shown below. If that's the case, what do you think is the difference between your code and Sophie's code that would result in this discrepancy? 

(Regardless of what happens with the code below, you will receive full credit for Problem 1b as long as all the functional requirements shown above are implemented correctly.)

In [6]:
# This line of code works:
# print(embeddings["the", "of"])

# This line of code doesn't work:
# print(embeddings["the"])

## Problem 2: Load Testing Data (4 Points in Total)

In this problem, you will load and deserialize the analogies dataset used to run the 3CosAdd test.

### Problem 2a: Inspect Data File (No Submission, 0 Points)

Please inspect the file `data/analogies.txt`, which contains all the analogies used in the 3CosAdd test. What is the format of this file? Notice that the analogies are organized into blocks. Each block contains analogies belonging to a different _relationship type_, and the first row of each block contains the name of the relationship type for that block.

### Problem 2b: Implement Data Loading Script (Code, 4 Points)

Next, you will implement the function `load_analogies` in the `test_analogies` module. Your code needs to parse the `data/analogies.txt` file and return the data in the form of a `dict`. 

To understand the format of this dict, please look at lines 43–47 of `test_analogies.py`.<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3) This part of the code defines the [type alias](https://docs.python.org/3/library/typing.html#type-aliases) `AnalogiesDataset`, which describes the format in which your code should organize the data. As shown in the following snippet, the keys of an `AnalogiesDataset` should be the names of all the relation types in the data file.

In [7]:
test_data = load_analogies("data/analogies.txt")
test_data.keys()

dict_keys(['capital-common-countries', 'capital-world', 'currency', 'city-in-state', 'family', 'gram1-adjective-to-adverb', 'gram2-opposite', 'gram3-comparative', 'gram4-superlative', 'gram5-present-participle', 'gram6-nationality-adjective', 'gram7-past-tense', 'gram8-plural', 'gram9-plural-verbs'])

For each relation type, the corresponding value in the `AnalogiesDataset` should be a list containing all the analogies under that relation type.

In [8]:
# Show the first 10 analogies for the relation type capital-world
for analogy in test_data["family"][:10]:
    print(analogy)

('boy', 'girl', 'brother', 'sister')
('boy', 'girl', 'brothers', 'sisters')
('boy', 'girl', 'dad', 'mom')
('boy', 'girl', 'father', 'mother')
('boy', 'girl', 'grandfather', 'grandmother')
('boy', 'girl', 'grandpa', 'grandma')
('boy', 'girl', 'grandson', 'granddaughter')
('boy', 'girl', 'groom', 'bride')
('boy', 'girl', 'he', 'she')
('boy', 'girl', 'his', 'her')


In [9]:
# Peek at the analogies under each relation type
for k, v in test_data.items():
    print("{}: {}, {} more analogies...".format(k, v[0], len(v) - 1))

capital-common-countries: ('Athens', 'Greece', 'Baghdad', 'Iraq'), 505 more analogies...
capital-world: ('Abuja', 'Nigeria', 'Accra', 'Ghana'), 4523 more analogies...
currency: ('Algeria', 'dinar', 'Angola', 'kwanza'), 865 more analogies...
city-in-state: ('Chicago', 'Illinois', 'Houston', 'Texas'), 2466 more analogies...
family: ('boy', 'girl', 'brother', 'sister'), 505 more analogies...
gram1-adjective-to-adverb: ('amazing', 'amazingly', 'apparent', 'apparently'), 991 more analogies...
gram2-opposite: ('acceptable', 'unacceptable', 'aware', 'unaware'), 811 more analogies...
gram3-comparative: ('bad', 'worse', 'big', 'bigger'), 1331 more analogies...
gram4-superlative: ('bad', 'worst', 'big', 'biggest'), 1121 more analogies...
gram5-present-participle: ('code', 'coding', 'dance', 'dancing'), 1055 more analogies...
gram6-nationality-adjective: ('Albania', 'Albanian', 'Argentina', 'Argentinean'), 1598 more analogies...
gram7-past-tense: ('dancing', 'danced', 'decreasing', 'decreased'), 

To receive full credit, the output of your function must satisfy the following requirements.
* The keys of the `dict` must exactly match the names of the relation types listed in the `data/analogies.txt` data file. The keys should not include the initial `: ` in the names of the relation types, and they should not contain any leading or trailing whitespace.
* Each analogy must be represented as a `tuple` or `list` of exactly 4 strings.

## Problem 3: Implement 3CosAdd (8 Points in Total)

For this problem, you will implement code that answers analogy questions and computes the analogy question accuracy attained by a set of word embeddings on each relation type.

### Problem 3a: Inspect test_analogies Module (No Submission, 0 Points)

You will now implement all the functions defined in the `test_analogies` module apart from `load_analogies`, which you should have implemented in Problem 2b. There are three functions in total:
* `cosine_sim`, which computes cosine similarity
* `get_closest_words`, which finds the closest words (in terms of cosine similarity) to a given vector in the embedding space
* `run_analogy_test`, which runs 3CosAdd on a set of word embeddings (represented by the `Embeddings` object).

### Problem 3b: Calculate Cosine Similarity (Code, 2 Points)

Please implement the function `cosine_sim`, referring to the docstring and the following code snippet for guidance. The function should take two matrices of row vectors and compute the cosine similarity between every row of the first matrix and every row of the second matrix. Recall that the cosine similarity between two vectors $\mathbf{u}$ and $\mathbf{v}$ is given by
$$ \cos(\mathbf{u}, \mathbf{v}) = \frac{\mathbf{u} \cdot \mathbf{v}}{\lVert \mathbf{u} \rVert \cdot \lVert \mathbf{v} \rVert}\text{;} $$
in other words, it is the dot product of $\mathbf{u}$ and $\mathbf{v}$ when they are normalized to unit length.

To receive full credit, your code must support the following usage. After line 8 is executed, `sims[i, j]` must contain the cosine similarity between `a[i]` and `b[j]` for each `i` and `j`.

In [10]:
a = np.array([[2, 5, 9, 10], 
              [0, -8, 3, -3]])
b = np.array([[-6, 7, 1, -3],
              [-3, 7, 9, 0],
              [-6, 10, 10, -5]])

# Full output of cosine_sim
sims = cosine_sim(a, b)
print("All Cosine Similarities:\n{}\n".format(sims))

# Rows correspond to rows of a, columns correspond to rows of b
for i, j in itertools.product(range(len(a)), range(len(b))):
    print("cos({}, {}) = {}".format(a[i], b[j], sims[i, j]))

All Cosine Similarities:
[[ 0.01415985  0.64383657  0.33316909]
 [-0.49852156 -0.27163371 -0.2392439 ]]

cos([ 2  5  9 10], [-6  7  1 -3]) = 0.014159846508095774
cos([ 2  5  9 10], [-3  7  9  0]) = 0.6438365650063621
cos([ 2  5  9 10], [-6 10 10 -5]) = 0.3331690892566788
cos([ 0 -8  3 -3], [-6  7  1 -3]) = -0.49852156261828706
cos([ 0 -8  3 -3], [-3  7  9  0]) = -0.2716337139226147
cos([ 0 -8  3 -3], [-6 10 10 -5]) = -0.23924389509855976


**Hint:** `cosine_sim` can be implemented in **at most 3 lines of code**. The performance of your code may suffer if it is substantially longer than this!

### Problem 3c: Find Neighboring Words (Code, 2 Points)

Next, please implement the function `get_closest_words`. This function finds the `k` closest words to one or more given vector(s) in the embedding space. The given vectors do not have to be valid word embeddings (though they do need to have the same dimension as the word embeddings), and the `k` closest words do not have to be in order.

For example, this is how you would find the 4 nearest neighbors of _king_, _man_, and _woman_ in the embedding space. (Note that each word is one of its own 4 nearest neighbors.)

In [11]:
try:  # Load embeddings if they haven't been loaded yet
    embeddings
except NameError:
    embeddings = Embeddings.from_file("word_embeddings/glove_50d.txt")
    
# Find the neighbors of "king," "man," and "woman"
vecs = embeddings["king", "man", "woman"]
get_closest_words(embeddings, vecs, k=4)

[['king', 'prince', 'queen', 'kingdom'],
 ['man', 'woman', 'person', 'another'],
 ['girl', 'woman', 'mother', 'man']]

The output above is interpreted as follows:
* the 4 nearest neighbors of _king_ are _ii_, _king_, _queen_, and _prince_
* the 4 nearest neighbors of _man_ are _man_, _woman_, _boy_, and _another_
* the 4 nearest neighbors of _woman_ are _woman_, _girl_, _man_, and _mother_.

Once you have implemented `get_closest_words`, you will use it to answer analogy questions. For instance, the analogy question for man : king :: woman : queen (if $\text{man} : \text{king} \mathrel{::} \text{woman} : x$, then what is $x$?) can be answered as follows.

In [12]:
# Find the neighbors of king - man + woman
queen = vecs[0:1] - vecs[1:2] + vecs[2:3]
get_closest_words(embeddings, queen, k=4)

[['king', 'queen', 'princess', 'prince']]

In this instance, we are being a bit more _lenient_ than Mikolov et al. (2013): we will consider the analogy question to have been answered correctly because $\overrightarrow{\text{queen}}$ is one of the **top 4 closest** word embeddings to $\overrightarrow{\text{king}} - \overrightarrow{\text{man}} + \overrightarrow{\text{woman}}$, even though Mikolov et al. required it to be **the closest** word embedding (which in case happens to be $\overrightarrow{\text{king}}$). 

**Hints:**
* `run_analogy_test` can be implemented in **at most 3 lines of code**.
* You can make your code run faster if you don't try to return the closest words in order (though the difference in performance may not be noticeable since we are using a small number of embeddings).

### Problem 3d: Write Testing Script (Code, 4 Points)

Finally, please implement the function `run_analogy_test`. Your code should run the 3CosAdd test on a given embedding space and testing dataset. The output of your code, illustrated below, should be a dict containing the analogy question accuracy for analogies from each relation type.

In [13]:
# Run the analogy test
# run_analogy_test(embeddings, test_data, k=2)

analogies = [
    ("france", "paris", "italy"),
    ("france", "paris", "japan"),
    ("france", "paris", "florida"),
    ("big", "bigger", "small"),
    ("big", "bigger", "cold"),
    ("big", "bigger", "quick"),
]

print(f"{'Analogy Question':40} {'GloVe 100 Prediction':20}")
print("-"*65)

for w1, w2, w3 in analogies:
    
    # 3CosAdd: b - a + c
    target = embeddings[[w2]] - embeddings[[w1]] + embeddings[[w3]]
    
    # Get closest word
    answer = get_closest_words(embeddings, target, k=1)[0][0]
    
    question_string = f"{w1} : {w2} :: {w3} : x"
    
    print(f"{question_string:40} {answer:20}")




Analogy Question                         GloVe 100 Prediction
-----------------------------------------------------------------
france : paris :: italy : x              rome                
france : paris :: japan : x              tokyo               
france : paris :: florida : x            florida             
big : bigger :: small : x                smaller             
big : bigger :: cold : x                 cold                
big : bigger :: quick : x                quick               


**Hint:** `run_analogy_test` can be implemented in **at most 9 lines of code**. The performance of your code may suffer if it is substantially longer than this! 

## Problem 4: Interpretation of Results (4 Points in Total)

In the final part of this assignment, you will use your code to study the three word embedding spaces provided in the assignment's GitHub repository. You may (optionally) wish to write additional code for the following problems, but you do not need to submit it.

### Problem 4a: Syntactic vs. Semantic Relation Types (Written, 2 Points)

In [Mikolov et al. (2013)](https://arxiv.org/abs/1301.3781), the 14 relation types are grouped into two categories: _semantic_ relation types and _syntactic_ relation types. Please look through the paper and find out which relation types are semantic and which are syntactic.

Now, please run the 3CosAdd test with a lenience of `k = 1` on all three embedding spaces (the 50-, 100-, and 200-dimensional embeddings), and then answer the following questions.
1. What is the total analogy question accuracy obtained by each embedding space for all analogies belonging belonging to a **semantic** relation type?
2. What is the total analogy question accuracy obtained by each embedding space for all analogies belonging belonging to a **syntactic** relation type?
3. What is the total analogy question accuracy obtained by each embedding space for **all analogies** in the testing dataset?
4. How do your GloVe results compare to the results for the two word2vec models (Skip-Gram and CBOW) reported in Table 4 of Mikolov et al. (2013)? Does the dimensionality of the embedding space have any effect on analogy question accuracy?

The only thing you need to submit for this problem is your answers to the above bullet points. For questions 1–3 you are encouraged to report your results in the form of a table such as the following. **Please report all numerical results with 3 significant figures of precision.**

| Embedding Space | Semantic | Syntactic | Overall |
|-----------------|----------|-----------|---------|
| GloVe 50        |          |           |         |
| GloVe 100       |          |           |         |
| GloVe 200       |          |           |         |

### Problem 4b: Effect of Lenience (Written, 1 Point)

Please repeat Problem 4a using a lenience of `k = 2`. Do the results change? If so, how?

### Problem 4c: Qualitative Evaluation (Written, 1 Point)

For the final problem, you will replicate Table 8 of Mikolov et al. (2013) by looking at some examples of answers to analogy questions. For each of the three embedding spaces, please answer the following analogy questions. 
* france : paris :: italy : $x$
* france : paris :: japan : $x$
* france : paris :: florida : $x$
* big : bigger :: small : $x$
* big : bigger :: cold : $x$
* big : bigger :: quick : $x$

You are encouraged to present your answer as a table like the following.

| Analogy Question                | Gold Answer  | GloVe 50 | GloVe 100 | GloVe 200 |
|---------------------------------|--------------|----------|-----------|-----------|
| france : paris :: italy : _x_   | rome         |          |           |           |
| france : paris :: japan : _x_   | tokyo        |          |           |           |
| france : paris :: florida : _x_ | tallahassee  |          |           |           |
| big : bigger :: small : _x_     | smaller      |          |           |           |
| big : bigger :: cold : _x_      | colder       |          |           |           |
| big : bigger :: quick : _x_     | quicker      |          |           |           |


Please comment on your results. How do the different embedding spaces compare to one another? How do they compare to the results reported by Mikolov et al.?

## Problem 5: Socialize (2 EC in Total)

### Problem 5a: Ask Your Friends for Help (No Submission, 1 Point Extra Credit)

Ask at least question on Piazza related to this assignment that is visible to all students in the class.

### Problem 5b: Help a Friend (No Submission, 1 Point Extra Credit)

Answer at least one question on Piazza related to this assignment that was asked by a student in the class.

## Footnotes

<a name="cite_note-1"></a>1. [<sup>^</sup>](#cite_ref-1) This terminology is due to [Drodz et al. (2016)](https://aclanthology.org/C16-1332/); Mikolov et al. (2013) call it the "vector offset method."

<a name="cite_note-2"></a>2. [<sup>^</sup>](#cite_ref-2) To make the assignment easier to download, the word embedding files have been truncated to a manageable size. The full word embedding files are available [here](https://nlp.stanford.edu/projects/glove/).

<a name="cite_note-3"></a>3. [<sup>^</sup>](#cite_ref-3) Obviously, these line numbers may be inaccurate if you have already started working on Problem 3.